In [1]:
#Biblitecas iniciais
import numpy as np
import pandas as pd

In [2]:
#Carregamento dos dados
df = pd.read_csv('https://raw.githubusercontent.com/wellingtondantas/Data-Science/master/Datasets/covid_pre_condition.csv')
df

,id,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
0,16169f,2,1,04-05-2020,02-05-2020,9999-99-99,97,2,27,97,2,2,2,2,2,2,2,2,2,2,2,1,97
1,1009bf,2,1,19-03-2020,17-03-2020,9999-99-99,97,2,24,97,2,2,2,2,2,2,2,2,2,2,99,1,97
2,167386,1,2,06-04-2020,01-04-2020,9999-99-99,2,2,54,2,2,2,2,2,2,2,2,1,2,2,99,1,2
3,0b5948,2,2,17-04-2020,10-04-2020,9999-99-99,2,1,30,97,2,2,2,2,2,2,2,2,2,2,99,1,2
4,0d01b5,1,2,13-04-2020,13-04-2020,22-04-2020,2,2,60,2,1,2,2,2,1,2,1,2,2,2,99,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566597,01ff60,2,1,13-05-2020,03-05-2020,9999-99-99,97,2,58,97,1,2,2,2,2,2,2,2,2,2,2,3,97
566598,047cd1,1,1,07-04-2020,06-04-2020,9999-99-99,97,2,48,2,2,2,2,2,2,2,2,2,2,2,2,3,97
566599,1beb81,1,2,14-05-2020,01-05-2020,9999-99-99,2,1,49,2,2,2,2,2,2,2,2,1,2,2,99,3,2
566600,16fb02,1,1,31-05-2020,29-05-2020,9999-99-99,97,1,43,2,2,2,2,2,2,2,2,2,2,2,2,3,97


In [3]:
#Transfere as informações para outra base
dfe = df

In [4]:
#Porém, na idade, realmente uma pessoa pode ter 97,98,99 anos
dfe.iloc[:,9:]=dfe.iloc[:,9:].replace({97:np.nan,98:np.nan,99:np.nan})
dfe=dfe.dropna()
dfe

,id,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
21,154fea,1,2,02-06-2020,02-06-2020,9999-99-99,2,2,25,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0
30,16e14e,1,2,22-06-2020,17-06-2020,9999-99-99,2,2,52,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0
71,149f1c,1,2,17-06-2020,12-06-2020,9999-99-99,2,1,51,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0
79,1d5df2,1,2,08-06-2020,07-06-2020,9999-99-99,1,1,67,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0
93,18f860,1,2,27-05-2020,27-05-2020,9999-99-99,2,1,59,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566497,70426,1,2,19-06-2020,12-06-2020,9999-99-99,2,1,62,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,3.0,2.0
566499,0143db,1,2,24-06-2020,21-06-2020,9999-99-99,2,2,35,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,3.0,2.0
566543,188cd4,1,2,06-06-2020,01-06-2020,9999-99-99,2,1,57,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,3.0,2.0
566559,1d5382,1,2,21-06-2020,16-06-2020,9999-99-99,2,2,17,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0


In [5]:
#Seleciona os features que mais incluenciam na análise
sintomas = dfe[['sex',
                'pneumonia',
                'age',
                'pregnancy',
                'diabetes',
                'copd',
                'asthma',
                'inmsupr',
                'hypertension',
                'other_disease',
                'cardiovascular', 
                'obesity',
                'renal_chronic',
                'tobacco']]

In [6]:
#Seleciona os pacientes que foram para a UTI e died
label = (dfe['date_died'] != '9999-99-99') | (dfe['icu'] == True)

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

In [8]:
#Classifica os sintomas com as label
gbt1 = GradientBoostingClassifier(random_state=0)
gbt1.fit(sintomas, label)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [9]:
#Extrair as informações importantes
importance1 = gbt1.feature_importances_

In [10]:
def get_importance(list_symptoms, importance):
    sym_imp_map = [] 
    for sym, imp in zip(list_symptoms, importance):
        sym_imp_map.append((sym,imp))
    return sym_imp_map 

In [11]:
symptom_imp_map1 = get_importance(sintomas.columns, importance1)

In [12]:
print('\n Fatores que mais influenciam no Obito ou Internação na UTI dos pacientes: \n')
sorted(symptom_imp_map1, reverse=True, key=lambda x: x[1])


 Fatores que mais influenciam no Obito ou Internação na UTI dos pacientes: 



[('pneumonia', 0.5375646181107002),
 ('age', 0.3359104608300066),
 ('diabetes', 0.037923676938741595),
 ('obesity', 0.02203161315179244),
 ('inmsupr', 0.015032586010777657),
 ('other_disease', 0.013516765232686887),
 ('hypertension', 0.01125861013504649),
 ('renal_chronic', 0.006453355195456402),
 ('cardiovascular', 0.0052038255952950975),
 ('asthma', 0.004915840512085642),
 ('tobacco', 0.004501081992572245),
 ('copd', 0.0036087533134355875),
 ('pregnancy', 0.0020788129814029318),
 ('sex', 0.0)]